In [18]:
import numpy as np

class Animal:
    
    # requried variables: needed for subclasses
    location = [[0], [0]]
    stepSize = 0
    mapSize = 0
    foodEaten = 0
    hunger = 0
    maxHunger = 0
    alive = True
    beStill = True
    
    def __init__(self, mapSize, location=None, stepSize = 1, maxHunger = 1000):
        
        if location == None:
            location = [np.random.randint(0, mapSize), np.random.randint(0, mapSize)]
        self.location = location
                
        self.stepSize = stepSize
        self.mapSize = mapSize
        self.foodEaten = 0
        self.hunger = 0
        self.alive = True
        self.beStill = False
        self.maxHunger = maxHunger
        
    def step(self):
        self.hunger = self.hunger + 1
        #move once for every stepSize
        for i in range(0, self.stepSize):
            
            #one for each direction
            direct = np.random.randint(0,8)
            
            # if the direction is 1,0,7 move x by +1
            if ((direct==0) or (direct==1) or (direct==7)):
                self.location[0] = self.location[0] + 1
                
            # if the direction is 1,2,3 move y by +1
            if ((direct==1) or (direct==2) or (direct==3)):
                self.location[1] = self.location[1] + 1
                
            # if the direction is 3,4,5 move x by -1
            if ((direct==3) or (direct==4) or (direct==5)):
                self.location[0] = self.location[0] - 1
                
            # if the direction is 5,6,7 move y by -1
            if ((direct==5) or (direct==6) or (direct==7)):
                self.location[1] = self.location[1] - 1
            
    # check if location needs to wrap
    def locationCheck(self):
        
        for i in range(0,2):
            if (self.location[i] > self.mapSize):
                self.location[i] = self.location[i] - self.mapSize
            elif (self.location[i] < 0):
                self.location[i] = self.mapSize - self.location[i]
    


In [19]:
class Food:
    location = []
    alive = True
    mapSize = 0
    
    def __init__(self, mapSize, location = None):
        
        if location == None:
            location = [np.random.randint(0, mapSize), np.random.randint(0, mapSize)]
        self.location = location
        
        self.mapSize = mapSize

In [20]:
    def hungerCheck(animalArray):
        
        everyoneDead = True
        for i in range(0, len(animalArray)):
            hunger = animalArray.hunger
            maxHunger = animalArray.maxHunger
            
            if hunger > maxHunger:
                animalArray(i).beStill = True
            else:
                everyoneDead = False
            return animalArray, everyoneDead